## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 29th Oct. 6th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,September-30-2024,Condo/Co-op,550 Okeechobee Blvd Unit Uph-02,West Palm Beach,FL,33401.0,1976500.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10994622,N,Y,26.705124,-80.056199


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
52,PAST SALE,2024-10-04,Condo/Co-op,201 S Narcissus Ave #1502,West Palm Beach,FL,33401.0,2980000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10996022,N,Y,26.711982,-80.051026
19,PAST SALE,2024-10-04,Condo/Co-op,2860 S Ocean Blvd #103,Palm Beach,FL,33480.0,583135.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2860-S-Oc...,Beaches MLS,RX-11020193,N,Y,26.616651,-80.039614
122,PAST SALE,2024-10-04,Condo/Co-op,12 Colonial Club Dr #105,Boynton Beach,FL,33435.0,272000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/12-Col...,Beaches MLS,RX-10918007,N,Y,26.502392,-80.059506
63,PAST SALE,2024-10-04,Condo/Co-op,1291 NW 13th St Unit 450-D,Boca Raton,FL,33486.0,275000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1291-NW-1...,Beaches MLS,F10449771,N,Y,26.362029,-80.112435
55,PAST SALE,2024-10-04,Condo/Co-op,2309 S Federal Hwy #17,Boynton Beach,FL,33435.0,143000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/2309-S...,MARMLS,A11627445,N,Y,26.508272,-80.060151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,PAST SALE,2024-09-30,Condo/Co-op,8862 Windtree St,Boca Raton,FL,33496.0,267000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/8862-Wind...,Beaches MLS,RX-11005190,N,Y,26.401138,-80.178858
54,PAST SALE,2024-09-30,Condo/Co-op,2868 Ashley Dr E Unit E,West Palm Beach,FL,33415.0,183000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2868...,Beaches MLS,RX-11008154,N,Y,26.635326,-80.117675
53,PAST SALE,2024-09-30,Condo/Co-op,9233 SW 8th St #122,Boca Raton,FL,33428.0,188000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/9233-SW-8...,Beaches MLS,RX-10998614,N,Y,26.339363,-80.190776
49,PAST SALE,2024-09-30,Condo/Co-op,4704 Lucerne Lakes Blvd #105,Lake Worth,FL,33467.0,257500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/4704-Luce...,Beaches MLS,RX-11002761,N,Y,26.604079,-80.156920


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

123


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$444,746


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$291


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$54,703,750


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
93
Input Previous Week Condo Average Sales Price:
497032
Input Previous Week Condo Average PSF:
289
Input Previous Week Condo Sales total (ex: 198_000_000)
45_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-01,Condo/Co-op,2720 Donald Ross Rd #507,Palm Beach Gardens,FL,33410.0,4095000.0,3.0,3.5,Azure Condo,3282.0,NaN,2018.0,NaN,1248.0,3330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-10981251,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Cristine Tamasi,Illustrated Properties LLC (Co,None,None,Dory Faxon,Compass Florida LLC,Geraldine Stanko,Compass Florida LLC,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-01,Condo/Co-op,2720 Donald Ross Rd #507,Palm Beach Gardens,FL,33410.0,4095000.0,3.0,3.5,Azure Condo,3282.0,NaN,2018.0,NaN,1248.0,3330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-10981251,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Cristine Tamasi,Illustrated Properties LLC (Co,None,None,Dory Faxon,Compass Florida LLC,Geraldine Stanko,Compass Florida LLC,1,orange
1,PAST SALE,2024-09-30,Condo/Co-op,350 S Ocean Blvd Unit 11d,Boca Raton,FL,33432.0,3300000.0,3.0,3.0,Beresford Cond,2523.0,NaN,1977.0,NaN,1308.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10986927,N,Y,26.347232,-80.070218,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Erin Omiatek,One Sotheby's International Realty,None,None,Senada Adzem,Douglas Elliman,None,None,2,blue
2,PAST SALE,2024-09-30,Condo/Co-op,3170 S Ocean Blvd Unit S405,Palm Beach,FL,33480.0,3200000.0,3.0,3.0,Enclave OF Palm Beach Condo,2950.0,NaN,1985.0,NaN,1085.0,3687.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Beaches MLS,RX-10913354,N,Y,26.605363,-80.037760,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Leslie Judge,Illustrated Properties,None,None,Nina Grody,William Raveis South Florida,None,None,3,blue
3,PAST SALE,2024-10-04,Condo/Co-op,201 S Narcissus Ave #1502,West Palm Beach,FL,33401.0,2980000.0,2.0,3.0,Esplanade Grande Condo,2434.0,NaN,2004.0,NaN,1224.0,2381.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10996022,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Miguel Rodriguez,Compass Florida LLC (PB),None,None,4,blue
4,PAST SALE,2024-10-01,Condo/Co-op,1177 George Bush Blvd Unit 308 A,Delray Beach,FL,33483.0,2620000.0,3.0,3.5,1177 Moderne,1978.0,NaN,2024.0,NaN,1325.0,2571.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Beaches MLS,RX-11015041,N,Y,26.474566,-80.059896,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Julie Ann Giachetti,One Sotheby's International Realty,None,None,Mary Beth Windle,Douglas Elliman,Caron Dockerty,Douglas Elliman,5,blue
5,PAST SALE,2024-09-30,Condo/Co-op,550 Okeechobee Blvd Unit Uph-02,West Palm Beach,FL,33401.0,1976500.0,3.0,2.0,Cityplace South Tower Condo,1606.0,NaN,2008.0,NaN,1231.0,1706.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10994622,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Jonathan Huffer,Redfin Corporation,Elena Fleck,Redfin Corporation,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",Michael Leibowitz,"Leibowitz Realty Group, LLC./PBG",6,blue
6,PAST SALE,2024-09-30,Condo/Co-op,110 Gleason St #202,Delray Beach,FL,33483.0,1837500.0,4.0,3.5,Bella Vista Condo,2105.0,NaN,2006.0,NaN,873.0,2229.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Beaches MLS,RX-11008404,N,Y,26.459182,-80.061011,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Judith Lukens,"Premier Estate Properties, Inc",Kim Lekas,"Premier Estate Properties, Inc",Patricia Burke,One Sotheby's International Realty,None,None,7,blue
7,PAST SALE,2024-09-30,Condo/Co-op,750 Ocean Royale Way #302,Juno Beach,FL,33408.0,1700000.0,3.0,3.0,Ocean Royale Condo,2643.0,NaN,2000.0,NaN,643.0,2900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/750-Ocean...,Beaches MLS,RX-10975016,N,Y,26.891689,-80.058145,https://www.redfin.com/FL/Juno-Beach/750-Ocean...,None,None,None,None,None,None,None,None,8,

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-01,Condo/Co-op,2720 Donald Ross Rd #507,Palm Beach Gardens,FL,33410.0,4095000.0,3.0,3.5,Azure Condo,3282.0,NaN,2018.0,NaN,1248.0,3330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-10981251,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Cristine Tamasi,Illustrated Properties LLC (Co,None,None,Dory Faxon,Compass Florida LLC,Geraldine Stanko,Compass Florida LLC,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_County_condo_sales_week_ending_10072024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'Azure Condo at 2720 Donald Ross Rd #507 in Palm Beach Gardens'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-01,Condo/Co-op,2720 Donald Ross Rd #507,Palm Beach Gardens,FL,33410.0,4095000.0,3.0,3.5,Azure Condo,3282.0,NaN,2018.0,NaN,1248.0,3330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-10981251,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Cristine Tamasi,Illustrated Properties LLC (Co,None,None,Dory Faxon,Compass Florida LLC,Geraldine Stanko,Compass Florida LLC,1,orange,Azure Condo at 2720 Donald Ross Rd #507 in Pal...
1,PAST SALE,2024-09-30,Condo/Co-op,350 S Ocean Blvd Unit 11d,Boca Raton,FL,33432.0,3300000.0,3.0,3.0,Beresford Cond,2523.0,NaN,1977.0,NaN,1308.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10986927,N,Y,26.347232,-80.070218,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Erin Omiatek,One Sotheby's International Realty,None,None,Senada Adzem,Douglas Elliman,None,None,2,blue,Beresford Cond at 350 S Ocean Blvd Unit 11d in...
2,PAST SALE,2024-09-30,Condo/Co-op,3170 S Ocean Blvd Unit S405,Palm Beach,FL,33480.0,3200000.0,3.0,3.0,Enclave OF Palm Beach Condo,2950.0,NaN,1985.0,NaN,1085.0,3687.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Beaches MLS,RX-10913354,N,Y,26.605363,-80.037760,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Leslie Judge,Illustrated Properties,None,None,Nina Grody,William Raveis South Florida,None,None,3,blue,Enclave OF Palm Beach Condo at 3170 S Ocean Bl...
3,PAST SALE,2024-10-04,Condo/Co-op,201 S Narcissus Ave #1502,West Palm Beach,FL,33401.0,2980000.0,2.0,3.0,Esplanade Grande Condo,2434.0,NaN,2004.0,NaN,1224.0,2381.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10996022,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Miguel Rodriguez,Compass Florida LLC (PB),None,None,4,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
4,PAST SALE,2024-10-01,Condo/Co-op,1177 George Bush Blvd Unit 308 A,Delray Beach,FL,33483.0,2620000.0,3.0,3.5,1177 Moderne,1978.0,NaN,2024.0,NaN,1325.0,2571.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Beaches MLS,RX-11015041,N,Y,26.474566,-80.059896,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Julie Ann Giachetti,One Sotheby's International Realty,None,None,Mary Beth Windle,Douglas Elliman,Caron Dockerty,Douglas Elliman,5,blue,1177 Moderne at 1177 George Bush Blvd Unit 308...
5,PAST SALE,2024-09-30,Condo/Co-op,550 Okeechobee Blvd Unit Uph-02,West Palm Beach,FL,33401.0,1976500.0,3.0,2.0,Cityplace South Tower Condo,1606.0,NaN,2008.0,NaN,1231.0,1706.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10994622,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Jonathan Huffer,Redfin Corporation,Elena Fleck,Redfin Corporation,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",Michael Leibowitz,"Leibowitz Realty Group, LLC./PBG",6,blue,Cityplace South Tower Condo at 550 Okeechobee ...
6,PAST SALE,2024-09-30,Condo/Co-op,110 Gleason St #202,Delray Beach,FL,33483.0,1837500.0,4.0,3.5,Bella Vista Condo,2105.0,NaN,2006.0,NaN,873.0,2229.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Beaches MLS,RX-11008404,N,Y,26.459182,-80.061011,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Judith Lukens,"Premier Estate Properties, Inc",Kim Lekas,"Premier Estate Properties, Inc",Patricia Burke,One Sotheby's International Realty,None,None,7,blue,Bella Vista Condo at 110 Gleason St #202 in De...

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Azure Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,360,000M to $4,095,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach, Juno Beach, West Palm Beach, Delray Beach, Palm Beach Gardens, Jupiter, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 123 condo sales totaling $54,703,750 million from Sep. 29th to Oct. 6th. The previous week, brokers closed 93 condo sales totaling $45,900,000.

Last week’s units sold for an average of $444,746, lower than

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-01,Condo/Co-op,2720 Donald Ross Rd #507,Palm Beach Gardens,FL,33410.0,4095000.0,3.0,3.5,Azure Condo,3282.0,NaN,2018.0,NaN,1248.0,3330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-10981251,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Cristine Tamasi,Illustrated Properties LLC (Co,None,None,Dory Faxon,Compass Florida LLC,Geraldine Stanko,Compass Florida LLC,1,orange,Azure Condo at 2720 Donald Ross Rd #507 in Pal...
1,PAST SALE,2024-09-30,Condo/Co-op,350 S Ocean Blvd Unit 11d,Boca Raton,FL,33432.0,3300000.0,3.0,3.0,Beresford Cond,2523.0,NaN,1977.0,NaN,1308.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10986927,N,Y,26.347232,-80.070218,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Erin Omiatek,One Sotheby's International Realty,None,None,Senada Adzem,Douglas Elliman,None,None,2,blue,Beresford Cond at 350 S Ocean Blvd Unit 11d in...
2,PAST SALE,2024-09-30,Condo/Co-op,3170 S Ocean Blvd Unit S405,Palm Beach,FL,33480.0,3200000.0,3.0,3.0,Enclave OF Palm Beach Condo,2950.0,NaN,1985.0,NaN,1085.0,3687.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Beaches MLS,RX-10913354,N,Y,26.605363,-80.037760,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Leslie Judge,Illustrated Properties,None,None,Nina Grody,William Raveis South Florida,None,None,3,blue,Enclave OF Palm Beach Condo at 3170 S Ocean Bl...
3,PAST SALE,2024-10-04,Condo/Co-op,201 S Narcissus Ave #1502,West Palm Beach,FL,33401.0,2980000.0,2.0,3.0,Esplanade Grande Condo,2434.0,NaN,2004.0,NaN,1224.0,2381.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10996022,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Miguel Rodriguez,Compass Florida LLC (PB),None,None,4,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
4,PAST SALE,2024-10-01,Condo/Co-op,1177 George Bush Blvd Unit 308 A,Delray Beach,FL,33483.0,2620000.0,3.0,3.5,1177 Moderne,1978.0,NaN,2024.0,NaN,1325.0,2571.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Beaches MLS,RX-11015041,N,Y,26.474566,-80.059896,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Julie Ann Giachetti,One Sotheby's International Realty,None,None,Mary Beth Windle,Douglas Elliman,Caron Dockerty,Douglas Elliman,5,blue,1177 Moderne at 1177 George Bush Blvd Unit 308...
5,PAST SALE,2024-09-30,Condo/Co-op,550 Okeechobee Blvd Unit Uph-02,West Palm Beach,FL,33401.0,1976500.0,3.0,2.0,Cityplace South Tower Condo,1606.0,NaN,2008.0,NaN,1231.0,1706.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10994622,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Jonathan Huffer,Redfin Corporation,Elena Fleck,Redfin Corporation,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",Michael Leibowitz,"Leibowitz Realty Group, LLC./PBG",6,blue,Cityplace South Tower Condo at 550 Okeechobee ...
6,PAST SALE,2024-09-30,Condo/Co-op,110 Gleason St #202,Delray Beach,FL,33483.0,1837500.0,4.0,3.5,Bella Vista Condo,2105.0,NaN,2006.0,NaN,873.0,2229.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Beaches MLS,RX-11008404,N,Y,26.459182,-80.061011,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Judith Lukens,"Premier Estate Properties, Inc",Kim Lekas,"Premier Estate Properties, Inc",Patricia Burke,One Sotheby's International Realty,None,None,7,blue,Bella Vista Condo at 110 Gleason St #202 in De...

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Azure Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,360,000M to $4,095,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach, Juno Beach, West Palm Beach, Delray Beach, Palm Beach Gardens, Jupiter, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 123 condo sales totaling $54,703,750 million from Sep. 29th to Oct. 6th. The previous week, brokers closed 93 condo sales totaling $45,900,000.

Last week’s units sold for an average of $444,746, lower than

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Palm-Beach-Gardens/2720-Donald-Ross-Rd-33410/unit-507/home/171497898


In [57]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/350-S-Ocean-Blvd-33432/unit-11d/home/190764723


In [50]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Jupiter/4161-S-US-Highway-1-33477/unit-M4/home/42411559


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/West-Palm-Beach/525-S-Flagler-Dr-33401/unit-19C/home/42519200


In [52]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Delray-Beach/1177-George-Bush-Blvd-33483/unit-308-A/home/189018580


In [53]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Juno-Beach/750-Ocean-Royale-Way-33408/unit-302/home/42548233


In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-01,Condo/Co-op,2720 Donald Ross Rd #507,Palm Beach Gardens,FL,33410.0,4095000.0,3.0,3.5,Azure Condo,3282.0,NaN,2018.0,NaN,1248.0,3330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-10981251,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Cristine Tamasi,Illustrated Properties LLC (Co,None,None,Dory Faxon,Compass Florida LLC,Geraldine Stanko,Compass Florida LLC,1,orange,Azure Condo at 2720 Donald Ross Rd #507 in Pal...
1,PAST SALE,2024-09-30,Condo/Co-op,350 S Ocean Blvd Unit 11d,Boca Raton,FL,33432.0,3300000.0,3.0,3.0,Beresford Cond,2523.0,NaN,1977.0,NaN,1308.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10986927,N,Y,26.347232,-80.070218,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Erin Omiatek,One Sotheby's International Realty,None,None,Senada Adzem,Douglas Elliman,None,None,2,blue,Beresford Cond at 350 S Ocean Blvd Unit 11d in...
2,PAST SALE,2024-09-30,Condo/Co-op,3170 S Ocean Blvd Unit S405,Palm Beach,FL,33480.0,3200000.0,3.0,3.0,Enclave OF Palm Beach Condo,2950.0,NaN,1985.0,NaN,1085.0,3687.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Beaches MLS,RX-10913354,N,Y,26.605363,-80.037760,https://www.redfin.com/FL/Palm-Beach/3170-S-Oc...,Leslie Judge,Illustrated Properties,None,None,Nina Grody,William Raveis South Florida,None,None,3,blue,Enclave OF Palm Beach Condo at 3170 S Ocean Bl...
3,PAST SALE,2024-10-04,Condo/Co-op,201 S Narcissus Ave #1502,West Palm Beach,FL,33401.0,2980000.0,2.0,3.0,Esplanade Grande Condo,2434.0,NaN,2004.0,NaN,1224.0,2381.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10996022,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Miguel Rodriguez,Compass Florida LLC (PB),None,None,4,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
4,PAST SALE,2024-10-01,Condo/Co-op,1177 George Bush Blvd Unit 308 A,Delray Beach,FL,33483.0,2620000.0,3.0,3.5,1177 Moderne,1978.0,NaN,2024.0,NaN,1325.0,2571.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Beaches MLS,RX-11015041,N,Y,26.474566,-80.059896,https://www.redfin.com/FL/Delray-Beach/1177-Ge...,Julie Ann Giachetti,One Sotheby's International Realty,None,None,Mary Beth Windle,Douglas Elliman,Caron Dockerty,Douglas Elliman,5,blue,1177 Moderne at 1177 George Bush Blvd Unit 308...
5,PAST SALE,2024-09-30,Condo/Co-op,550 Okeechobee Blvd Unit Uph-02,West Palm Beach,FL,33401.0,1976500.0,3.0,2.0,Cityplace South Tower Condo,1606.0,NaN,2008.0,NaN,1231.0,1706.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10994622,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Jonathan Huffer,Redfin Corporation,Elena Fleck,Redfin Corporation,Andrew Leibowitz,"Leibowitz Realty Group, LLC./PBG",Michael Leibowitz,"Leibowitz Realty Group, LLC./PBG",6,blue,Cityplace South Tower Condo at 550 Okeechobee ...
6,PAST SALE,2024-09-30,Condo/Co-op,110 Gleason St #202,Delray Beach,FL,33483.0,1837500.0,4.0,3.5,Bella Vista Condo,2105.0,NaN,2006.0,NaN,873.0,2229.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Beaches MLS,RX-11008404,N,Y,26.459182,-80.061011,https://www.redfin.com/FL/Delray-Beach/110-Gle...,Judith Lukens,"Premier Estate Properties, Inc",Kim Lekas,"Premier Estate Properties, Inc",Patricia Burke,One Sotheby's International Realty,None,None,7,blue,Bella Vista Condo at 110 Gleason St #202 in De...

## Time on Market Calculator

In [60]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 5, 24) ## List (Earlier) date
date2 = datetime(2024, 10, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

130


## Clean CSV for Datawrapper Chart

In [61]:
chart_df = df_top_ten

In [62]:
chart_df = chart_df.fillna(" ")

In [63]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [64]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [65]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [66]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [67]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [68]:
csv_date_string = today.strftime("%m_%d_%Y")

In [69]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")